# Insurance Prediction for Production

In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib

In [2]:
# Load saved components
components = joblib.load("best_Insurance_Prediction_model.joblib")
model = components['model']
scaler = components['scaler']
selected_features = components['selected_features']

In [3]:
# Define categorical mappings 
exercise_map = {'Never': 0, 'Rarely': 1, 'Occasionally': 2, 'Frequently': 3}
coverage_map = {'Basic': 0, 'Standard': 1, 'Premium': 2}

In [4]:
# Define categorical options for Gradio inputs
gender_options = ["male", "female"]
smoker_options = ["yes", "no"]
region_options = ["northeast", "northwest", "southeast", "southwest"]
exercise_options = list(exercise_map.keys())
coverage_options = list(coverage_map.keys())
medical_history_options = ["None", "Diabetes", "Heart Disease", "Other"]
family_history_options = ["None", "Diabetes", "Heart Disease", "Other"]
occupation_options = ["Tech", "Business", "Healthcare", "Other"]

In [5]:
# Prediction function
def predict(age, bmi, children, gender, smoker, region, exercise_frequency,
            coverage_level, medical_history, family_medical_history, occupation):
    
    # Create input dataframe
    data = {
        "age": [age],
        "bmi": [bmi],
        "children": [children],
        "gender": [gender],
        "smoker": [smoker],
        "region": [region],
        "exercise_frequency": [exercise_frequency],
        "coverage_level": [coverage_level],
        "medical_history": [medical_history],
        "family_medical_history": [family_medical_history],
        "occupation": [occupation]
    }
    df = pd.DataFrame(data)
    
    # One-hot encode categorical columns (same as training)
    df_encoded = pd.get_dummies(df, columns=['gender','smoker','region','medical_history',
                                             'family_medical_history','occupation'],
                                drop_first=True, dtype=int)
    
    # Apply ordinal mapping
    df_encoded['exercise_frequency'] = df_encoded['exercise_frequency'].map(exercise_map)
    df_encoded['coverage_level'] = df_encoded['coverage_level'].map(coverage_map)
    
    # Ensure all selected features exist
    for col in selected_features:
        if col not in df_encoded.columns:
            df_encoded[col] = 0  # add missing columns as 0
    
    # Scale numerical columns
    scale_cols = ['age','bmi','children','exercise_frequency','coverage_level']
    df_encoded[scale_cols] = scaler.transform(df_encoded[scale_cols])
    
    # Reorder columns to match training
    df_encoded = df_encoded[selected_features]
    
    # Predict
    prediction = model.predict(df_encoded)[0]
    
    return f"Predicted Insurance Charges: ${prediction:.2f}"


In [6]:
# Build Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## Insurance Prediction App")
    
    with gr.Row():
        age = gr.Number(label="Age", value=30)
        bmi = gr.Number(label="BMI", value=25.0)
        children = gr.Number(label="Number of Children", value=0)
    
    with gr.Row():
        gender = gr.Dropdown(gender_options, label="Gender")
        smoker = gr.Dropdown(smoker_options, label="Smoker")
        region = gr.Dropdown(region_options, label="Region")
    
    with gr.Row():
        exercise_frequency = gr.Dropdown(exercise_options, label="Exercise Frequency")
        coverage_level = gr.Dropdown(coverage_options, label="Coverage Level")
    
    with gr.Row():
        medical_history = gr.Dropdown(medical_history_options, label="Medical History")
        family_medical_history = gr.Dropdown(family_history_options, label="Family Medical History")
        occupation = gr.Dropdown(occupation_options, label="Occupation")
    
    predict_btn = gr.Button("Predict")
    output = gr.Textbox(label="Prediction Result")
    
    predict_btn.click(fn=predict, 
                      inputs=[age,bmi,children,gender,smoker,region,
                              exercise_frequency,coverage_level,medical_history,
                              family_medical_history,occupation],
                      outputs=output)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
